In [2]:
from read_input import *
import codecs

# Frequency input
Put together the letter and bigram frequency files based on the data that comes from Mathieu. Distributes the frequency of combined characters that are not in the to-be-mapped character set of the scenario and filters out characters that are irrelevant.

In [37]:
#scenario="scenario1"
#derive_probabilities_from_raw_values(letter_file, bigram_file, scenario)

#scenario="scenario2"
#derive_probabilities_from_raw_values(letter_file, bigram_file, scenario)

scenario="scenario3_a"
letter_file = "input/resources/Code_all_Avg_freqs_1.tsv"
bigram_file = "input/resources/Code_all_Avg_freqs_2.tsv"
derive_probabilities_from_raw_values(letter_file, bigram_file, scenario, "_code")

letter_file = "input/resources/Twitter_all_Avg_freqs_1.tsv"
bigram_file = "input/resources/Twitter_all_Avg_freqs_2.tsv"
derive_probabilities_from_raw_values(letter_file, bigram_file, scenario, "_twitter")

letter_file = "input/resources/Formals_all_Avg_freqs_1.tsv"
bigram_file = "input/resources/Formals_all_Avg_freqs_2.tsv"
derive_probabilities_from_raw_values(letter_file, bigram_file, scenario, "_formal")



[u'!', u'#', u'$', u'%', u'&', u'(', u')', u'*', u'+', u',', u',d', u'-', u'-d', u'.', u'.d', u'/', u'/d', u':', u';', u'<', u'=', u'>', u'?', u'@', u'[', u'\\', u']', u'^d', u'_', u'_d', u'{', u'|', u'}', u'~d', u'\xa1', u'\xa2', u'\xa3', u'\xa4', u'\xa5', u'\xa7', u'\xa8d', u'\xa9', u'\xaa', u'\xab', u'\xae', u'\xb0', u'\xb0d', u'\xb1', u'\xb4d', u'\xb7', u'\xb8d', u'\xba', u'\xbb', u'\xbc', u'\xbd', u'\xbe', u'\xbf', u'\xc6', u'\xd0', u'\xd7', u'\xde', u'\xdf', u'\xe0', u'\xe6', u'\xe7', u'\xe8', u'\xe9', u'\xf0', u'\xf7', u'\xf9', u'\xfe', u'\u0130', u'\u0131', u'\u0132', u'\u0133', u'\u014a', u'\u014b', u'\u0152', u'\u0153', u'\u017f', u'\u018f', u'\u01b7', u'\u0259', u'\u0292', u'\u02c7d', u'\u02c9d', u'\u02cbd', u'\u02d8d', u'\u02d9d', u'\u02dbd', u'\u02ddd', u'\u02f5d', u'\u0311d', u'\u03b82', u'\u1e9e', u'\u2011', u'\u2013', u'\u2014', u'\u2018', u'\u2019', u'\u201a', u'\u201c', u'\u201d', u'\u201e', u'\u2020', u'\u2021', u'\u2026', u'\u2030', u'\u2039', u'\u203a', u'\u20a9', 

({u'!': 0.00067807630104057398,
  u'#': 4.0054146333371398e-05,
  u'$': 7.9827195808396071e-06,
  u'%': 0.00022123029739333372,
  u'&': 9.1661225505604643e-05,
  u'(': 0.0015574668345407167,
  u')': 0.0017116214028072305,
  u'*': 0.00044893875334910078,
  u'+': 6.6616313260024082e-05,
  u',': 0.0095089599925427103,
  u',d': 1.3126400914200265e-06,
  u'-': 0.0026215493914986776,
  u'-d': 1.2553492637463219e-09,
  u'.': 0.0082053695472610114,
  u'.d': 5.708558374219645e-07,
  u'/': 0.00056749660849826525,
  u'/d': 2.1784704403716231e-10,
  u':': 0.0013507667832056826,
  u';': 0.00049261353498268622,
  u'<': 5.4913224789460306e-06,
  u'=': 5.1660213422725304e-05,
  u'>': 2.7737492943705361e-05,
  u'?': 0.00046253532014866965,
  u'@': 3.7824739613119998e-05,
  u'[': 9.1058126342147784e-05,
  u'\\': 7.1959569789152299e-06,
  u']': 0.00012457483673867484,
  u'^d': 0.002019072757524889,
  u'_': 0.00030273557802304376,
  u'_d': 4.2801532012094497e-08,
  u'a': 0.060795404589578363,
  u'b': 0.00

### Single letter ranking based on probability, corpora weighted

In [11]:

corpus_weights = {"Formal": 0.5, "Twitter":0.3, "Code":0.2}

letter_files = ["input/resources/Formals_all_Avg_freqs_1.tsv","input/resources/Code_all_Avg_freqs_1.tsv", "input/resources/Twitter_all_Avg_freqs_1.tsv"]
p_single={}
p_bigrams = {}
for l_file in letter_files:
    print(l_file)
    print("weighting corpora")
    single = pd.read_csv(l_file, sep=" ", encoding="utf-8", index_col=0, quoting=3)
    single = single.to_dict()[u'frequency']
    #weight according to given weight:
    weight = 0
    for k,v in corpus_weights.iteritems():
        if k in l_file:
            weight = v
            break;
    if weight == 0:
        raise ValueError('no weight found for file: %s'%l_file)

    if len(p_single) == 0:
        p_single = {c:v*weight for c,v in single.iteritems()}     
    else:
        for c,v in single.iteritems():
            if c in p_single:
                p_single[c]+= v*weight
            else:
                p_single[c] = v*weight

input/resources/Formals_all_Avg_freqs_1.tsv
weighting corpora
input/resources/Code_all_Avg_freqs_1.tsv
weighting corpora
input/resources/Twitter_all_Avg_freqs_1.tsv
weighting corpora


In [34]:
df = pd.DataFrame.from_dict(p_single, orient="index")
df = df.reset_index()
df.columns=["letter", "frequency"]
df = df.sort_values(by="frequency", ascending=False)
with codecs.open("lists/letter_order_all.txt", 'w', encoding="utf-8") as f:
    for i in range(0,len(df)):
        f.write("%s\t%.8f\n"%(df.iloc[i,0], df.iloc[i,1]))
#df.to_csv("lists/letter_order_all.txt", sep="\t", encoding="utf-8", index=False)


# Distances
You only need to rerun this if you need to change the level cost

In [3]:
level_cost = {u'':0, u'Shift':1, u'Alt':2, u'Alt_Shift':3}

distance0, distance1 = read_distance_matrix(level_cost, recompute=0)
lines = ["%s %s %f\n"%(k1,k2,d) for (k1,k2), d in distance0.iteritems()]
with codecs.open("input\\distance0.txt", 'w', encoding="utf-8") as f:
    f.writelines(lines)
    
lines = ["%s %s %f\n"%(k1,k2,d) for (k1,k2), d in distance1.iteritems()]
with codecs.open("input\\distance1.txt", 'w', encoding="utf-8") as f:
    f.writelines(lines)